# 1 Scraping sobre los post de páginas de Facebook

## falla sobre como web crapping sobre barack Obama pero funciona como uno parcial:

Fuente:

https://minimaxir.com/2015/07/facebook-scraper/

https://github.com/minimaxir/facebook-page-post-scraper/blob/master/examples/how_to_build_facebook_scraper.ipynb

How to Download a Facebook Page Posts and Comments to Excel with Python:

https://onlinezhuanjia.com/python-facebook-page-scraper-tool/

El acceso a los datos de la página de Facebook requiere un token de acceso.
Dado que el token de acceso de usuario caduca dentro de una hora, necesitamos crear una aplicación ficticia con el único propósito de raspar y usar el ID de la aplicación y el secreto de la aplicación generados allí como se describe aquí, los cuales nunca caducan.

https://developers.facebook.com/docs/facebook-login/access-tokens#apptokens

IDENTIFICADOR DE LA APP: 615060279226779

Token de cliente:        e2828d07bab87d46d5edcf8df49e798c

API facebook: https://facebook-sdk.readthedocs.io/en/latest/api.html#class-facebook-graphapi

In [ ]:
import json
import datetime
import csv
import time
try:
    from urllib.request import urlopen, Request
except ImportError:
    from urllib2 import urlopen, Request
 
# app_id = "615060279226779"
# app_secret = "a3b9cd27faf240ff46b6509c15089a77"
page_id = "BarackObama"
 
# input date formatted as YYYY-MM-DD
since_date = "2018-01-01"
until_date = "2018-01-30"
 
# access_token = app_id + "|" + app_secret
access_token = "615060279226779|Fi9fxsS8onNd73aOIFg1r47CfTs"
 
 
def request_until_succeed(url):
    req = Request(url)
    success = False
    while success is False:
        try:
            response = urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception as e:
            print(e)
            time.sleep(5)
 
            print("Error for URL {}: {}".format(url, datetime.datetime.now()))
            print("Retrying.")
 
    return response.read()
 
 
# Needed to write tricky unicode correctly to csv
def unicode_decode(text):
    try:
        return text.encode('utf-8').decode()
    except UnicodeDecodeError:
        return text.encode('utf-8')
 
 
def getFacebookPageFeedUrl(base_url):
 
    # Construct the URL string; see http://stackoverflow.com/a/37239851 for
    # Reactions parameters
    fields = "&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id," + \
        "comments.limit(0).summary(true),shares,reactions" + \
        ".limit(0).summary(true)"
 
    return base_url + fields
 
 
def getReactionsForStatuses(base_url):
 
    reaction_types = ['like', 'love', 'wow', 'haha', 'sad', 'angry']
    reactions_dict = {}   # dict of {status_id: tuple&amp;amp;amp;lt;6&amp;amp;amp;gt;}
 
    for reaction_type in reaction_types:
        fields = "&amp;amp;amp;amp;fields=reactions.type({}).limit(0).summary(total_count)".format(
            reaction_type.upper())
 
        url = base_url + fields
 
        data = json.loads(request_until_succeed(url))['data']
 
        data_processed = set()  # set() removes rare duplicates in statuses
        for status in data:
            id = status['id']
            count = status['reactions']['summary']['total_count']
            data_processed.add((id, count))
 
        for id, count in data_processed:
            if id in reactions_dict:
                reactions_dict[id] = reactions_dict[id] + (count,)
            else:
                reactions_dict[id] = (count,)
 
    return reactions_dict
 
 
def processFacebookPageFeedStatus(status):
 
    # The status is now a Python dictionary, so for top-level items,
    # we can simply call the key.
 
    # Additionally, some items may not always exist,
    # so must check for existence first
 
    status_id = status['id']
    status_type = status['type']
 
    status_message = '' if 'message' not in status else \
        unicode_decode(status['message'])
    link_name = '' if 'name' not in status else \
        unicode_decode(status['name'])
    status_link = '' if 'link' not in status else \
        unicode_decode(status['link'])
    status_permalink_url = '' if 'permalink_url' not in status.keys() else \
            unicode_decode(status['permalink_url'])
 
    # Time needs special care since a) it's in UTC and
    # b) it's not easy to use in statistical programs.
 
    status_published = datetime.datetime.strptime(
        status['created_time'], '%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + \
        datetime.timedelta(hours=-5)  # EST
    status_published = status_published.strftime(
        '%Y-%m-%d %H:%M:%S')  # best time format for spreadsheet programs
 
    # Nested items require chaining dictionary keys.
 
    num_reactions = 0 if 'reactions' not in status else \
        status['reactions']['summary']['total_count']
    num_comments = 0 if 'comments' not in status else \
        status['comments']['summary']['total_count']
    num_shares = 0 if 'shares' not in status else status['shares']['count']
 
    return (status_id, status_message, link_name, status_type, status_link,status_permalink_url,
            status_published, num_reactions, num_comments, num_shares)
 
 
def scrapeFacebookPageFeedStatus(page_id, access_token, since_date, until_date):
    with open('{}_facebook_statuses.csv'.format(page_id), 'w',encoding="utf-8") as file:
        w = csv.writer(file)
        w.writerow(["status_id", "status_message", "link_name", "status_type",
                    "status_link","permalink_url", "status_published", "num_reactions",
                    "num_comments", "num_shares", "num_likes", "num_loves",
                    "num_wows", "num_hahas", "num_sads", "num_angrys",
                    "num_special"])
 
        has_next_page = True
        num_processed = 0
        scrape_starttime = datetime.datetime.now()
        after = ''
        base = "https://graph.facebook.com/v2.9"
        node = "/{}/posts".format(page_id)
        parameters = "/?limit={}&amp;amp;amp;amp;access_token={}".format(100, access_token)
        since = "&amp;amp;amp;amp;since={}".format(since_date) if since_date \
            is not '' else ''
        until = "&amp;amp;amp;amp;until={}".format(until_date) if until_date \
            is not '' else ''
 
        print("Scraping {} Facebook Page: {}\n".format(page_id, scrape_starttime))
 
        while has_next_page:
            after = '' if after is '' else "&amp;amp;amp;amp;after={}".format(after)
            base_url = base + node + parameters + after + since + until
 
            url = getFacebookPageFeedUrl(base_url)
            statuses = json.loads(request_until_succeed(url))
            reactions = getReactionsForStatuses(base_url)
 
            for status in statuses['data']:
 
                # Ensure it is a status with the expected metadata
                if 'reactions' in status:
                    status_data = processFacebookPageFeedStatus(status)
                    reactions_data = reactions[status_data[0]]
 
                    # calculate thankful/pride through algebra
                    num_special = status_data[7] - sum(reactions_data)
                    w.writerow(status_data + reactions_data + (num_special,))
 
                num_processed += 1
                if num_processed % 100 == 0:
                    print("{} Statuses Processed: {}".format
                          (num_processed, datetime.datetime.now()))
 
            # if there is no next page, we're done.
            if 'paging' in statuses:
                after = statuses['paging']['cursors']['after']
            else:
                has_next_page = False
 
        print("\nDone!\n{} Statuses Processed in {}".format(
              num_processed, datetime.datetime.now() - scrape_starttime))
 
 
if __name__ == '__main__':
    scrapeFacebookPageFeedStatus(page_id, access_token, since_date, until_date)

Scraping BarackObama Facebook Page: 2020-03-20 12:13:51.114165

HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47CfTs&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-01-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 12:13:56.364009
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47CfTs&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-01-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 12:14:01.649620
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/Barack

Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47CfTs&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-01-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 12:17:14.969797
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47CfTs&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-01-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 12:17:20.178598
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47C

Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47CfTs&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-01-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 12:20:33.398658
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47CfTs&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-01-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 12:20:38.608591
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47C

Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47CfTs&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-01-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 12:23:51.978902
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47CfTs&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-01-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 12:23:57.198939
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47C

Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47CfTs&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-01-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 12:27:10.361496
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47CfTs&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-01-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 12:27:15.562269
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47C

...

Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47CfTs&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-01-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 13:13:28.824254
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47CfTs&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-01-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 13:13:34.046003
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47C

Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47CfTs&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-01-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 13:16:47.196163
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47CfTs&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-01-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 13:16:52.415381
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47C

Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47CfTs&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-01-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 13:20:05.414273
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47CfTs&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-01-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 13:20:10.651771
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47C

Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47CfTs&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-01-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 13:23:23.719580
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47CfTs&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-01-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 13:23:28.930438
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47C

Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47CfTs&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-01-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 13:26:42.246042
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47CfTs&amp;amp;amp;amp;since=2018-01-01&amp;amp;amp;amp;until=2018-01-30&amp;amp;amp;amp;fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true): 2020-03-20 13:26:47.455817
Retrying.
HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.9/BarackObama/posts/?limit=100&amp;amp;amp;amp;access_token=615060279226779|Fi9fxsS8onNd73aOIFg1r47C

In [50]:

if __name__ == "__main__":
 
    logging.basicConfig(level=logging.INFO)
    base_url = 'https://mobile.facebook.com'
    session = requests.session()
 
    # Extracts credentials for the login and all of the profiles URL to scrape
    credentials = json_to_obj('credentials.json')
    profiles_urls = json_to_obj('profiles_urls.json')
 
    make_login(session, base_url, credentials)
 
    posts_data = None
    for profile_url in profiles_urls:
        posts_data = crawl_profile(session, base_url, profile_url, 25)
    logging.info('[!] Scraping finished. Total: {}'.format(len(posts_data)))
    logging.info('[!] Saving.')
    save_data(posts_data)
 

NameError: name 'logging' is not defined

In [7]:
# https://pypi.org/project/facebook-scraper/
# Se pueden extraer los post de cualquier página de 
# Facebook sin necesidad de autenticación alguna.
from facebook_scraper import get_posts
for post in get_posts('sudaustral', pages=10):
    print(post['text'][:500])
   # print(post['likes'][:500])

REDDGuatemala
December 6, 2019 at 9:13 PM ·
#SNICC Profesionales del MARN, Ministerio de Agricultura, Ganadería y Alimentación - MAGA, Conap Consejo Nacional de Áreas Protegidas, INAB Guatemala, #UVG, #SEGEPLAN y de los proyectos implementadores REDD+ han participando en uno de los cuatro módulos de capacitación programados sobre la temática “Herramientas informáticas para la visualización y reporte de información”.
La capacitación se realiza en el marco del proceso de transferencias tecnológica
REDDGuatemala
December 10, 2019 at 9:22 AM ·
🌳 Hace algunas semanas realizamos un Foro de Género, importante actividad dentro del proceso de consolidación de la Estrategia REDD+ Guatemala. Participaron lideresas y asociaciones ambientales para la presentación de avances en la incorporación de las consideraciones de género. #Bosque #Gente #Futuro
REDDGuatemala
December 11, 2019 at 12:23 PM Â·
ð³ De acuerdo al proceso de ConsolidaciÃ³n de la Estrategia REDD+ Guatemala, realizamos un Foro de GÃ©

¡Todas y todos trabajando por 
REDDGuatemala
August 7, 2019 at 7:14 PM ·
🗣 Ayer tuvimos jornada de capacitación con el Grupo Interinstitucional de Monitoreo de Bosque y Uso de la Tierra (GIMBUT) para analizar la ruta de trabajo en perspectiva de la actualización del Nivel de Referencia de Emisiones y Absorciones Forestales.
¡Todos y todas trabajando por el futuro de nuestros bosques! #Guatemala
Queremos compartir con ustedes esta entrevista que fue realizada a nuestro Director Patricio Emanuelli en marco de la Estrategia REDD+ Guatemala, encargada por el Banco Interamericano de Desarrollo (BID) al Consorcio Sud-Austral - #GOPA - #CALMECAC - #ForestFinest
¡Trabajando por el futuro de nuestros bosques!
REDDGuatemala
August 5, 2019 at 10:00 PM ·
🗣 Hoy tuvimos una jornada de capacitación para el equipo técnico que fortalecerá las mancomunidades del Corredor Seco, en perspectiva de la articulación de la Estrategia Nacional REDD+ Guatemala con la planificación territorial. #Bosque #Gente #Fu

In [4]:
import facebook
graph = facebook.GraphAPI(access_token = 'EAADUCx9H1b4BAKgLQlodIERBVdOshlEJDdyKJ58SrievGHa6ZAZCUq0ZAg1jzJdmJiJElGYyqANim6nhKh2iwvYZBfdi2MXJXeo6Bd098j0X5bsMnVb9DK09XhPvobv5d30KrjBkaf2u4PAQI2GBAbZAIaLBKO2z3mz0krHNXuAZDZD', version="2.12")
graph.get_object(id = 'me', fields = 'id, name')

ACCESS_TOKEN = 'EAADUCx9H1b4BAKgLQlodIERBVdOshlEJDdyKJ58SrievGHa6ZAZCUq0ZAg1jzJdmJiJElGYyqANim6nhKh2iwvYZBfdi2MXJXeo6Bd098j0X5bsMnVb9DK09XhPvobv5d30KrjBkaf2u4PAQI2GBAbZAIaLBKO2z3mz0krHNXuAZDZD'


In [5]:
import requests # pip install requests
import json

base_url = 'https://graph.facebook.com/me'

# Specify which fields to retrieve
fields = 'id,name,likes.limit(10){about}'

url = '{0}?fields={1}&access_token={2}'.format(base_url, fields, ACCESS_TOKEN)

# This API is HTTP-based and could be requested in the browser,
# with a command line utlity like curl, or using just about
# any programming language by making a request to the URL.
# Click the hyperlink that appears in your notebook output
# when you execute this code cell to see for yourself...
print(url)

# Interpret the response as JSON and convert back
# to Python data structures
content = requests.get(url).json()

# Pretty-print the JSON and display it
print(json.dumps(content, indent=1))

https://graph.facebook.com/me?fields=id,name,likes.limit(10){about}&access_token=EAADUCx9H1b4BAKgLQlodIERBVdOshlEJDdyKJ58SrievGHa6ZAZCUq0ZAg1jzJdmJiJElGYyqANim6nhKh2iwvYZBfdi2MXJXeo6Bd098j0X5bsMnVb9DK09XhPvobv5d30KrjBkaf2u4PAQI2GBAbZAIaLBKO2z3mz0krHNXuAZDZD
{
 "id": "121447789451413",
 "name": "Christian Castro"
}


In [6]:
import facebook # pip install facebook-sdk
import json

# A helper function to pretty-print Python objects as JSON
def pp(o):
    print(json.dumps(o, indent=1))

# Create a connection to the Graph API with your access token
g = facebook.GraphAPI(ACCESS_TOKEN, version='2.10')

# Execute a few example queries:

# Get my ID
pp(g.get_object('me'))

{
 "name": "Christian Castro",
 "id": "121447789451413"
}


In [7]:


# Get the connections to an ID
# Example connection names: 'feed', 'likes', 'groups', 'posts'
pp(g.get_connections(id='me', connection_name='likes'))



{
 "data": []
}


In [8]:
# Search for a location, may require approved app
pp(g.request("search", {'type': 'place', 'center': '40.749444, -73.968056', 'fields': 'name, location'}))

GraphAPIError: (#33) This object does not exist or does not support this action

In [9]:
# Search for a page's ID by name
pp(g.request("search", {'q': 'Mining the Social Web', 'type': 'page'}))

# Grab the ID for the book and check the number of fans
mtsw_id = '146803958708175'
pp(g.get_object(id=mtsw_id, fields=['fan_count']))

{
 "data": []
}


GraphAPIError: (#100) Page Public Metadata Access requires either app secret proof or an app token

In [42]:
# Retrieve information about a website or page:
site_info = graph.get_object(id="https%3A//mobolic.com",fields="og_object")
print(site_info["og_object"]["description"])

Mobolic is a web consultancy located in Washington DC.


https://elfsight.com/blog/2017/10/how-to-get-facebook-access-token/
Mi token:    
EAADUCx9H1b4BAKgLQlodIERBVdOshlEJDdyKJ58SrievGHa6ZAZCUq0ZAg1jzJdmJiJElGYyqANim6nhKh2iwvYZBfdi2MXJXeo6Bd098j0X5bsMnVb9DK09XhPvobv5d30KrjBkaf2u4PAQI2GBAbZAIaLBKO2z3mz0krHNXuAZDZD
233144234530238

https://facebook-sdk.readthedocs.io/en/latest/api.html
    
    Token de la NASA:
        233144234530238|7Ubaqd1zZDWOwj7K1Ag1Illdbcg

In [48]:
 # Write 'Hello, world' to the active user's wall.
 graph.put_object(parent_object='me', connection_name='feed',
                  message='Hello, world')

# Add a link and write a message about it.
graph.put_object(
   parent_object="me",
   connection_name="feed",
   message="This is a great website. Everyone should visit it.",
   link="https://www.facebook.com")

 # Write a comment on a post.
 graph.put_object(parent_object='post_id', connection_name='comments',
                  message='First!')


GraphAPIError: (#200) If posting to a group, requires app being installed in the group, and \
          either publish_to_groups permission with user token, or both manage_pages \
          and publish_pages permission with page token; If posting to a page, \
          requires both manage_pages and publish_pages as an admin with \
          sufficient administrative permission

In [ ]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re

#driver = webdriver.Firefox()
# 通知の無効化
options = webdriver.FirefoxOptions()
options.set_preference("dom.push.enabled", False)
driver = webdriver.Firefox(firefox_options=options)
driver.implicitly_wait(10)

url = "https://www.facebook.com"
driver.get(url)

email_elem = driver.find_element_by_name("email")
email_elem.send_keys("メールアドレス")
password_elem = driver.find_element_by_name("pass")
password_elem.send_keys("パスワード")
password_elem.submit()
time.sleep(2)

# 検索
input_keyword = driver.find_element_by_name("q")
input_keyword.send_keys("codecamp")
input_keyword.submit()
time.sleep(2)

# CodeCampの項目を確定
link = driver.find_element_by_link_text("CodeCamp")
link.click()
time.sleep(2)

# 自動スクロールダウン
#page = driver.find_element_by_tag_name("html")
#page.send_keys(Keys.END)

# 自動スクロール繰り返し
i = 1
scroll = 120

while i < scroll:
    try:
        print("\nループ開始：" + str(i) + "回目")
        page = driver.find_element_by_tag_name("html")
        page.send_keys(Keys.END)
        time.sleep(1.5)

    except:
        print("error...")

    finally:
        #element = driver.find_elements_by_class_name("_4-u2 _4-u8")
        #element = len(element)
        html = driver.page_source
        post = html.count("_4-u2 _4-u8")
        #print(html[0:200])
        print("ループOK: " + str(i) + "回目")
        print("投稿数:" + str(post))
        #file = open("facebook-html-" + str(i) + ".html", "w")
        #file.write(html)
        #file.close()
        
        i = i + 1

print("【ループ終了】" )
print("記事数:" + str(post))

x = re.findall(r'\brel="dialog">\w+', html)
print(x)
print("シェアのあった投稿数：" + str(len(x)))

# CSV用にデータ加工　1次元配列を 2次元配列に変換
import numpy as np
item = np.array(x)
item_count = len(x)
x = item.reshape(item_count,1)
print(x)
np.savetxt("fb_data.csv", x, delimiter = ",", fmt = "%s")